In [21]:
!pip install mailchimp-marketing



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
!pip install requests



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install requests schedule



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import requests
import csv
import hashlib
import os
import time

# Mailchimp Configuration (NOTE: In production, use environment variables instead)
api_key = 'fbae16afb0a35355b37499caf244bd6a-us16'
audience_id = '705d2651f6'
sender_email = 'j.d.barros@outlook.com'
sender_name = 'J.D. Barros'
csv_file_path = r'C:\Users\Barros\Downloads\Safari2.1.csv'

# SECURITY WARNING: In production, you should:
# 1. Revoke this API key after use
# 2. Use environment variables instead of hardcoded credentials
# 3. Never commit API keys to version control

# Extract Mailchimp datacenter from API key
try:
    datacenter = api_key.split('-')[-1]
    base_url = f'https://{datacenter}.api.mailchimp.com/3.0'
except (AttributeError, IndexError):
    raise ValueError("Invalid Mailchimp API key format")

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

def get_member_hash(email):
    """Generate MD5 hash for email address"""
    return hashlib.md5(email.lower().encode('utf-8')).hexdigest()

def upsert_contact(row, retries=3):
    """Add or update contact in Mailchimp with retry logic"""
    email = row.get('email', '').strip()
    if not email:
        print("Skipping row with missing email")
        return None

    member_hash = get_member_hash(email)
    url = f'{base_url}/lists/{audience_id}/members/{member_hash}'
    
    data = {
        'email_address': email,
        'status_if_new': 'subscribed',
        'merge_fields': {
            'FNAME': row.get('name', '').strip(),
            'COMPANY': row.get('company', '').strip()
        }
    }
    
    for attempt in range(retries):
        try:
            response = requests.put(url, headers=headers, json=data, timeout=30)
            if response.status_code in [200, 201, 204]:
                return response
            elif response.status_code == 429:  # Rate limited
                wait_time = int(response.headers.get('Retry-After', 10))
                print(f"Rate limited. Waiting {wait_time} seconds...")
                time.sleep(wait_time)
                continue
            else:
                print(f"Error with {email}: {response.status_code} - {response.text}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for {email}: {str(e)}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                return None

def upload_contacts():
    """Upload contacts from CSV with comprehensive error handling"""
    if not os.path.exists(csv_file_path):
        print(f"Error: CSV file not found at {csv_file_path}")
        return False

    success_count = 0
    error_count = 0
    skipped_count = 0
    
    try:
        with open(csv_file_path, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            total_rows = sum(1 for _ in reader)  # Count total rows
            file.seek(0)  # Reset file pointer
            next(reader)  # Skip header
            
            print(f"\nStarting upload of {total_rows} contacts...")
            
            for i, row in enumerate(reader, 1):
                if not row.get('email'):
                    skipped_count += 1
                    continue
                    
                print(f"Processing {i}/{total_rows}: {row.get('email')}", end='\r')
                
                response = upsert_contact(row)
                if response:
                    success_count += 1
                else:
                    error_count += 1

        print(f"\n\nUpload complete:")
        print(f"✅ Success: {success_count}")
        print(f"❌ Errors: {error_count}")
        print(f"⏩ Skipped: {skipped_count}")
        
        return error_count == 0
    except Exception as e:
        print(f"\nFatal error processing CSV: {str(e)}")
        return False

def create_and_send_campaign():
    """Create and send a campaign with HTML content"""
    campaign_subject = "Explore the Wonders of Kenya"
    
    html_content = """<!DOCTYPE html>
<html>
<head>
    <title>Kenyan Safari Adventure</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.6; max-width: 600px; margin: 0 auto; }
        .header { background-color: #f5f5f5; padding: 20px; text-align: center; }
        .content { padding: 20px; }
        .footer { background-color: #f5f5f5; padding: 10px; text-align: center; font-size: 0.8em; }
    </style>
</head>
<body>
    <div class="header">
        <h1>Discover Kenya's Wildlife</h1>
    </div>
    
    <div class="content">
        <p>Dear Adventurer,</p>
        <p>Experience the breathtaking beauty of Kenya's national parks with our exclusive safari packages.</p>
        
        <h2>Featured Destinations:</h2>
        <ul>
            <li>Maasai Mara National Reserve</li>
            <li>Amboseli National Park</li>
            <li>Tsavo East & West</li>
        </ul>
        
        <p>Special offers available for bookings this season!</p>
    </div>
    
    <div class="footer">
        <p>{unsubscribe} • {email}</p>
        <p>© 2023 My Kenyan Safari. All rights reserved.</p>
    </div>
</body>
</html>"""

    try:
        # Create campaign
        campaign_data = {
            'type': 'regular',
            'recipients': {'list_id': audience_id},
            'settings': {
                'subject_line': campaign_subject,
                'from_name': sender_name,
                'reply_to': sender_email,
                'title': f"Safari Campaign {int(time.time())}",
                'auto_footer': True,
                'inline_css': True
            }
        }

        print("\nCreating campaign...")
        create_response = requests.post(
            f'{base_url}/campaigns',
            headers=headers,
            json=campaign_data
        )
        create_response.raise_for_status()
        campaign_id = create_response.json()['id']
        print(f"Campaign created: {campaign_id}")

        # Set content
        print("Adding HTML content...")
        content_response = requests.put(
            f'{base_url}/campaigns/{campaign_id}/content',
            headers=headers,
            json={'html': html_content}
        )
        content_response.raise_for_status()

        # Send campaign
        print("Sending campaign...")
        send_response = requests.post(
            f'{base_url}/campaigns/{campaign_id}/actions/send',
            headers=headers
        )
        
        if send_response.status_code == 204:
            print("🎉 Campaign sent successfully!")
            return True
        else:
            print(f"Failed to send campaign: {send_response.text}")
            return False

    except requests.exceptions.RequestException as e:
        print(f"API Error: {str(e)}")
        return False
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False

def main():
    print("\n" + "="*50)
    print("MAILCHIMP AUTOMATION TOOL")
    print("="*50 + "\n")
    
    # Step 1: Upload contacts
    print("[1/2] UPLOADING CONTACTS")
    if not upload_contacts():
        print("\n⚠️ Contact upload had errors. Check output above.")
    
    # Step 2: Create and send campaign
    print("\n[2/2] CREATING CAMPAIGN")
    if not create_and_send_campaign():
        print("\n⚠️ Campaign creation failed. Check output above.")
    
    print("\nOperation completed.")

if __name__ == "__main__":
    main()


MAILCHIMP AUTOMATION TOOL

[1/2] UPLOADING CONTACTS

Starting upload of 1 contacts...
Processing 1/1: kchess13@gmail.com

Upload complete:
✅ Success: 1
❌ Errors: 0
⏩ Skipped: 0

[2/2] CREATING CAMPAIGN

Creating campaign...
Campaign created: 8b071c2e5a
Adding HTML content...
Sending campaign...
🎉 Campaign sent successfully!

Operation completed.


In [65]:
import requests
import csv
import hashlib
import time

# Mailchimp Configuration (using your hardcoded values)
api_key = 'fbae16afb0a35355b37499caf244bd6a-us16'
audience_id = '705d2651f6'
sender_email = 'j.d.barros@outlook.com'
sender_name = 'J.D. Barros'
csv_file_path = r'C:\Users\Barros\Downloads\Safari2.1.csv'

# SECURITY NOTE: In production, use environment variables instead of hardcoding credentials

# Validate API key format
if not api_key or '-' not in api_key:
    raise ValueError("Invalid Mailchimp API key format. Expected format: 'key-datacenter'")

# Extract datacenter and setup base URL
try:
    datacenter = api_key.split('-')[-1]
    base_url = f'https://{datacenter}.api.mailchimp.com/3.0'
except Exception as e:
    raise ValueError(f"Error processing API key: {str(e)}")

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

def get_member_hash(email):
    """Generate MD5 hash for email address"""
    return hashlib.md5(email.lower().encode('utf-8')).hexdigest()

def upsert_contact(row):
    """Add or update contact in Mailchimp"""
    email = row.get('email', '').strip()
    if not email:
        print("Skipping row with missing email")
        return None

    member_hash = get_member_hash(email)
    url = f'{base_url}/lists/{audience_id}/members/{member_hash}'
    
    data = {
        'email_address': email,
        'status_if_new': 'subscribed',
        'merge_fields': {
            'FNAME': row.get('name', '').strip(),
            'COMPANY': row.get('company', '').strip()
        }
    }
    
    try:
        response = requests.put(url, headers=headers, json=data, timeout=30)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {email}: {str(e)}")
        return None

def upload_contacts():
    """Upload contacts from CSV"""
    try:
        with open(csv_file_path, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for i, row in enumerate(reader, 1):
                response = upsert_contact(row)
                if response and response.status_code in [200, 201]:
                    print(f"{i}. Success: {row.get('email')}")
                else:
                    print(f"{i}. Error: {row.get('email')} - {response.status_code if response else 'Connection failed'}")
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file_path}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")

def create_and_send_campaign():
    """Create and send campaign"""
    try:
        # Create campaign
        campaign_data = {
            'type': 'regular',
            'recipients': {'list_id': audience_id},
            'settings': {
                'subject_line': "Explore the Wonders of Kenya",
                'from_name': sender_name,
                'reply_to': sender_email,
                'title': "Kenyan Safari Campaign",
                'auto_footer': True
            }
        }

        response = requests.post(f'{base_url}/campaigns', headers=headers, json=campaign_data)
        response.raise_for_status()
        campaign_id = response.json()['id']
        print(f"Campaign created: {campaign_id}")

        # Set content
        html_content = "<html><body><h1>Kenyan Safari Adventure</h1></body></html>"
        requests.put(
            f'{base_url}/campaigns/{campaign_id}/content',
            headers=headers,
            json={'html': html_content}
        ).raise_for_status()

        # Send campaign
        send_response = requests.post(f'{base_url}/campaigns/{campaign_id}/actions/send', headers=headers)
        if send_response.status_code == 204:
            print("Campaign sent successfully!")
        else:
            print(f"Failed to send campaign: {send_response.text}")
    except requests.exceptions.RequestException as e:
        print(f"API Error: {str(e)}")

if __name__ == "__main__":
    print("1. Uploading contacts...")
    upload_contacts()
    
    print("\n2. Creating and sending campaign...")
    create_and_send_campaign()
    
    print("\nProcess completed!")

1. Uploading contacts...
1. Success: kchess13@gmail.com
2. Success: thesynergygroup0@gmail.com

2. Creating and sending campaign...
Campaign created: fb34614593
Campaign sent successfully!

Process completed!
